# Low-Level Design (LLD) of a Tic Tac Toe Game System

### 1. Requirements

1. **Game Mechanics**:
    - The game should allow two players to take turns marking a 3x3 grid with either 'X' or 'O' until a player wins or the board fills up (resulting in a draw).

2. **Player Turn Handling**:
    - The game should alternate between two players (Player 1 and Player 2).

3. **Game End:**:
    - The game ends when one player wins or when the board is full (draw).

4. **Board Display**:
    - The board should be printed after every move to show the current state.

5. **Winning Conditions**:
    - The game should check for win conditions after every move.

6. **Restart Game**:
    - Option to restart the game after a match is over.
    
---

### 2. Constraints

1. The game must operate on a 3x3 grid
2. Players must alternate turns, starting with Player 1 ('X').
3. Players can only place their mark on empty spots on the board.
4. No option to undo a move once made.
5. There are a maximum of 9 moves in total before a draw condition.

---

### 3. Identify Entities

1. **Game**:
    - Represents the entire game, including managing turns, checking for a win, and displaying the board.
    - Coordinates between the Player, Board, and Game Logic.

2. **Player**:
    - Represents a player (either Player 1 or Player 2) with attributes like name and symbol ('X' or 'O').
    - Responsible for making moves on the board.

3. **Board**:
    - Manages the game board and its current state.
    - Handles the placement of marks on the grid and checks the game state.

4. **GameLogic**:
    - Contains the rules and logic for checking win conditions, draw conditions, and determining whose turn it is.

5. **UserInterface**:
    - Handles displaying the game board, taking player input, and showing game status messages (e.g., "Player X wins!", "It's a draw").

### 4. Class Design

#### 4.1. Player

In [5]:
class Player:
    def __init__(self, name: str, symbol: str, player_id: int):
        self.name = name
        self.symbol = symbol
        self.player_id = player_id

    def make_move(self, board, row: int, col: int):
        if board.is_valid_move(row, col):
            board.grid[row][col] = self.symbol
            return True

#### 4.2. Board

In [7]:
class Board:
    def __init__(self, size: int = 3):
        self.size = size
        self.grid = [[' ' for _ in range(size)] for _ in range(size)]

    def display_board(self):
        for row in self.grid:
            print('|'.join(row))
            print('-' * (self.size * 2 - 1))

    def is_valid_move(self, row: int, col: int) -> bool:
        if not (0 <= row < self.size and 0 <= col < self.size):
            raise OutOfBoundsException()

        if self.grid[row][col] == ' ':
            return True

        raise InvalidMoveException()

    def check_winner(self):
        # Check rows and columns
        for i in range(self.size):
            if all(self.grid[i][0] == self.grid[i][j] != ' ' for j in range(self.size)):
                return self.grid[i][0]

            if all(self.grid[0][i] == self.grid[j][i] != ' ' for j in range(self.size)):
                return self.grid[0][i]

        # Check diagonals
        if all(self.grid[i][i] == self.grid[0][0] != ' ' for i in range(self.size)):
            return self.grid[0][0]

        if all(self.grid[i][self.size - i - 1] == self.grid[0][self.size - 1] != ' ' for i in range(self.size)):
            return self.grid[0][self.size - 1]

        return None  # No winner yet

    def is_full(self):
        return all(cell != ' ' for row in self.grid for cell in row)

    def reset_board(self):
        self.grid = [[' ' for _ in range(self.size)] for _ in range(self.size)]

#### 4.3. Game Class

In [9]:
class Game:
    def __init__(self, player1: Player, player2: Player, size: int = 3):
        self.players = [player1, player2]
        self.board = Board(size)
        self.current_player = player1
        self.game_over = False

    def switch_turn(self):
        self.current_player = self.players[1] if self.current_player == self.players[0] else self.players[0]

    def restart_game(self):
        self.board.reset_board()
        self.current_player = self.players[0]
        self.game_over = False
        print("\nGame restarted!\n")
        self.start_game()

    def start_game(self):
        while not self.game_over:
            self.board.display_board()
            try:
                row, col = map(int, input(f"{self.current_player.name} ({self.current_player.symbol}), enter row and col: ").split(','))

                if self.game_over:
                    raise GameOverException()

                self.current_player.make_move(self.board, row, col)
                winner = self.board.check_winner()

                if winner:
                    self.board.display_board()
                    print(f"🎉 {self.current_player.name} ({winner}) wins! 🎉")
                    self.game_over = True
                elif self.board.is_full():
                    self.board.display_board()
                    print("🤝 It's a draw! 🤝")
                    self.game_over = True
                else:
                    self.switch_turn()
            except (InvalidMoveException, OutOfBoundsException, ValueError) as e:
                print(e)

        # Now, ask if players want to restart
        self.ask_restart()

    def ask_restart(self):
        while True:
            choice = input("🔄 Do you want to restart the game? (yes/no): ").strip().lower()
            if choice in ['yes', 'y']:
                self.restart_game()
                break
            elif choice in ['no', 'n']:
                print("👋 Thank you for playing! Goodbye!")
                break
            else:
                print("❌ Invalid input. Please enter 'yes' or 'no'.")

#### 4.4. GameManager Class

In [11]:
class GameManager:
    def __init__(self):
        self.games = []

    def create_game(self, player1_name, player2_name):
        player1 = Player(player1_name, 'X', 1)
        player2 = Player(player2_name, 'O', 2)
        game = Game(player1, player2)
        self.games.append(game)
        return game

    def list_games(self):
        return [f"Game {i+1}: {game.players[0].name} vs {game.players[1].name}" for i, game in enumerate(self.games)]

#### 5. Exception Handling

In [13]:
class TicTacToeException(Exception):
    """Base Exception class for Tic Tac Toe."""
    pass


class InvalidMoveException(TicTacToeException):

    def __init__(self, message="Invalid move! Cell is already occupied or out of bounds."):
        super().__init__(message)


class OutOfBoundsException(TicTacToeException):

    def __init__(self, message="Move out of bounds! Please enter a valid row and column."):
        super().__init__(message)


class GameOverException(TicTacToeException):

    def __init__(self, message="Game is already over! No more moves allowed."):
        super().__init__(message)

#### 6. Implementation

In [15]:
if __name__ == "__main__":
    manager = GameManager()
    game = manager.create_game("Alice", "Bob")
    game.start_game()


 | | 
-----
 | | 
-----
 | | 
-----


Alice (X), enter row and col:  0,0


X| | 
-----
 | | 
-----
 | | 
-----


Bob (O), enter row and col:  1,1


X| | 
-----
 |O| 
-----
 | | 
-----


...

X|O|X
-----
X|O|O
-----
 | | 
-----


Alice (X), enter row and col:  2,0


X|O|X
-----
X|O|O
-----
X| | 
-----
🎉 Alice (X) wins! 🎉


🔄 Do you want to restart the game? (yes/no):  no


👋 Thank you for playing! Goodbye!
